In [ ]:
import numpy as np
import xarray as xr
import rasterio
from scipy.interpolate import griddata
from datetime import datetime
import os

# 读取极端降水指数数据
extreme_pr_path = 'G:\\lizhi_4090\\D\\lizhi\\guangdong_extreme_pre\\extreme_precipitation_indices_seasonal.nc'
extreme_pr_ds = xr.open_dataset(extreme_pr_path)

lat = extreme_pr_ds['lat'].values
lon = extreme_pr_ds['lon'].values
seasons = extreme_pr_ds['season'].values
years = np.arange(2000, 2023)  # 2000-2022年

# 定义广东略大一些的经纬度范围
lat_min, lat_max = 19, 30
lon_min, lon_max = 105, 120

# 读取并裁剪人口数据
def read_and_clip_population_data(year):
    pop_year = int(year)
    pop_path = f'G:\\lizhi_4090\\D\\CMIP6extreme\\00 - CN051-2022\\extreme_pr\\人口暴露\\landscan-global-{pop_year}-colorized.tif'
    
    if not os.path.exists(pop_path):
        raise FileNotFoundError(f"Population data for year {pop_year} not found at {pop_path}")

    with rasterio.open(pop_path) as src:
        pop_data = src.read(1, masked=True)
        transform = src.transform
        pop_lon, pop_lat = np.meshgrid(
            np.arange(pop_data.shape[1]) * transform[0] + transform[2],
            np.arange(pop_data.shape[0]) * transform[4] + transform[5]
        )
        # 裁剪到指定范围
        mask = (pop_lat >= lat_min) & (pop_lat <= lat_max) & (pop_lon >= lon_min) & (pop_lon <= lon_max)
        pop_lat = pop_lat[mask]
        pop_lon = pop_lon[mask]
        pop_data = pop_data[mask]
    return pop_lat, pop_lon, pop_data

# 插值人口数据到极端降水数据的网格上
def interpolate_population_to_precip_grid(pop_lat, pop_lon, pop_data, precip_lat, precip_lon):
    points = np.array([pop_lon.flatten(), pop_lat.flatten()]).T
    grid_x, grid_y = np.meshgrid(precip_lon, precip_lat)
    grid_data = griddata(points, pop_data.flatten(), (grid_x, grid_y), method='linear')
    return grid_data

# 初始化暴露矩阵
exposure_indices = ['R20', 'PRCPTOT', 'R10']
pop_exposure = {index: np.zeros((len(years), len(seasons), len(lat), len(lon)), dtype=np.float32) for index in exposure_indices}

for i, year in enumerate(years):
    print(f'Processing year: {year}')
    pop_lat, pop_lon, pop_data = read_and_clip_population_data(year)
    interpolated_pop = interpolate_population_to_precip_grid(pop_lat, pop_lon, pop_data, lat, lon)

    # 读取当前年份的极端降水指数数据，并转换为浮点数
    for index in exposure_indices:
        for season_idx, season in enumerate(seasons):
            if season == 'DJF':
                # 若是冬季，使用前一年的数据
                precip_year = year - 1
            else:
                precip_year = year

            precip_data = extreme_pr_ds[index].sel(year=precip_year, season=season).values.astype(np.float32)
            pop_exposure[index][i, season_idx, :, :] = precip_data * interpolated_pop

# 创建Xarray Dataset并保存为NetCDF文件
output_filename = 'population_exposure_2000_2022_adjusted.nc'
ds = xr.Dataset(
    {index: (('year', 'season', 'lat', 'lon'), pop_exposure[index]) for index in exposure_indices},
    coords={
        'year': years,
        'season': seasons,
        'lat': lat,
        'lon': lon
    }
)
ds.year.attrs['units'] = 'year'
ds.season.attrs['units'] = 'season'
ds.lat.attrs['units'] = 'degrees_north'
ds.lon.attrs['units'] = 'degrees_east'
for index in exposure_indices:
    ds[index].attrs['units'] = 'people_exposed'

ds.attrs['title'] = 'Population Exposure to Extreme Precipitation (Seasonal, 2000-2022, Adjusted Winter)'
ds.attrs['history'] = f'Created {datetime.utcnow().isoformat()}'

ds.to_netcdf(output_filename)
print(f'Population exposure data for 2000-2022 has been saved to {output_filename}')